In [38]:
import sys
import json
import os
from bs4 import BeautifulSoup

In [39]:
sys.version

'3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [40]:
def remove_unnecessary(name, soup):
    for s in soup.find_all(name):
        s.extract()

In [41]:
def add_to_dict(dictionary, name, tag, soup):
#     If nothing in the dictionary, add it.
    if(dictionary[name] == ''):
        dictionary[name] = []
    objs = soup.find_all(tag)
    if(len(objs) > 0):
        for obj in objs:
#             s = obj.text.strip().replace('\n', '').replace('\t', ' ')
            s = obj.text
            if(len(s) > 0):
                dictionary[name].append(s)
            obj.extract()

In [42]:
def parse_data(soup):
    # Script and Style are meaningless
    remove_unnecessary('script', soup)
    remove_unnecessary('style', soup)

    # Define the dictionary content and the items in it.
    # Currently we define it as:
    '''
    Link_Name: the text show on link name, since in that link it is more relevant, we should store it here
    Paragraph: the paragraphs in the page, it is the important things
    Title: I just stored the h1, h2, h3, Strong tags here, they are more important content.
    Span: The ones in span, they are not important, I just store them and extract them later.
    Others: The other content left in the page, it should have less weight.
    '''
    content = {}
    content['Link_Name'] = []
    content['Paragraph'] = []
    content['Title'] = []
    content['Span'] = []
    content['Others'] = []

    add_to_dict(content, 'Link_Name', 'a', soup)

    add_to_dict(content, 'Link_Name', 'li', soup)

    add_to_dict(content, 'Link_Name', 'option', soup)

    add_to_dict(content, 'Span', 'span', soup)

    add_to_dict(content, 'Paragraph', 'p', soup)

    for tag in ('title', 'h1', 'h2', 'h3', 'Strong'):
        add_to_dict(content, 'Title', tag, soup)
    try:
        soup.find('head').extract()
    except:
        None

#         Here since I extract all the read text above, the rest of them will not be duplicated.
#         And if there is any BROKEN html, it will read into "Others"
    for word in soup.text.split('\n'):
        word = word.strip()
        if(len(word) > 0):
            content['Others'].append(word)
            
    return content

In [43]:
with open("/Users/weijiaxu/Desktop/2019W/Information Retrieval/Project 3/GIT/Broken Demo/2") as f:
    soup1 = BeautifulSoup(f.read())
with open("/Users/weijiaxu/Desktop/2019W/Information Retrieval/Project 3/GIT/Broken Demo/2 broken") as f:
    soup2 = BeautifulSoup(f.read())

In [44]:
soup1

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>Arthur Asuncion's Official Site: Blog</title>
</head>
<body>
<p> This tag is with complete tag</p>
<p> This should be in a P </p>
<p> This should be in a P </p>
<p> This should be in a P </p>
</body>
</html>

In [45]:
soup2

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>Arthur Asuncion's Official Site: Blog</title>
</head>
<body>
This is a sentence with out any tags.
<p> This should be in a P </p>
This should be in a P without head
<p> This should be in a P without end
</p></body>
</html>

# Firstly, we choose BeautifulSoup since it can handle the broken html.
# Second, we add a mechanism to grip the rest text of the html and store it in "Others", this will also be helpful.

In [46]:
content1 = parse_data(soup1)

In [47]:
content2 = parse_data(soup2)

In [48]:
content1

{'Link_Name': [],
 'Paragraph': [' This tag is with complete tag',
  ' This should be in a P ',
  ' This should be in a P ',
  ' This should be in a P '],
 'Title': ["Arthur Asuncion's Official Site: Blog"],
 'Span': [],
 'Others': []}

In [49]:
content2

{'Link_Name': [],
 'Paragraph': [' This should be in a P ',
  ' This should be in a P without end\n'],
 'Title': ["Arthur Asuncion's Official Site: Blog"],
 'Span': [],
 'Others': ['This is a sentence with out any tags.',
  'This should be in a P without head']}